In [1]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
# GPU 사용 시
device = torch.device("cuda:0")

In [4]:
# kobert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


# Dataset

Huggingface에서 뉴스 기사 데이터를 다운로드함

In [5]:
import datasets
dataset1 = datasets.load_dataset('nlpHakdang/aihub-news30k',  data_files={"train":"news_train_1_1.csv", "valid":"news_valid_1_1.csv"}, use_auth_token='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')

Using custom data configuration nlpHakdang___aihub-news30k-1ea472361ab2489f
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___aihub-news30k-1ea472361ab2489f/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/2 [00:00<?, ?it/s]

종합 분야 기사는 제외하고 경제, 기업, IT/과학 관련 기사에 라벨 1을 주고 나머지는 0을 줘서 분류하도록 라벨링함

In [6]:
# from tqdm import tqdm

# modified_dset = []
# for news in tqdm(dataset1['train']):
#     if news['category'] == '종합':
#         continue
#     temp = []
#     temp.append(news['article'])
#     if news['category'] == '경제' or news['category'] == '기업' or news['category'] == 'IT,과학':
#          temp.append('1')
#     else:
#          temp.append('0')
#     modified_dset.append(temp)

In [7]:
from collections import Counter
Counter(dataset1['train']['category'])

Counter({'종합': 197022,
         '스포츠': 5455,
         '정치': 18007,
         '사회': 19649,
         '경제': 27286,
         '교육/입시/NIE': 1190,
         '선거': 2,
         '기업': 17,
         '보건/의료': 55,
         '부동산': 74,
         '북한/한반도정세': 5,
         'IT,과학': 2331})

In [8]:
from tqdm import tqdm

modified_dset = []
for news in tqdm(dataset1['train']):
    if news['category'] == '종합':
        continue
    temp = []
    temp.append(news['article'])
    if news['category'] == '경제' or news['category'] == 'IT,과학':
        temp.append('1')
    else:
        temp.append('0')
    modified_dset.append(temp)

100%|██████████| 271093/271093 [00:39<00:00, 6840.69it/s]


In [9]:
modified_dset

[['전남드래곤즈 해맞이 다짐 선수 영입 활발이성훈 전남드래곤즈 사장 신승재 는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다.임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를 달성하기 위한 결연한 의지를 다졌다.이번 해맞이 행사는 2018년을 시작하면서 떠오르는 해를 보며 전남드래곤즈 구성원 모두가 한마음 한 뜻으로 구단 목표 달성을 위해 정진하자는 의미에서 실시한 것이다.신승재 사장은 유상철 감독을 비롯한 코칭스텝 선수단 구성이 마무리 된 만큼 구성원 모두가 하나되어 K리그 클래식 5위 이내 진입 FA컵 우승 등 ACL 진출권 획득을 목표로 최선을 다하자 고 선수들에게 신년 인사말을 전했다.유상철 감독은 구봉산의 정기를 받아 2018년을 전남드래곤즈의 해로 만들겠다 며 각오를 다졌다.한편 전남은 선수들도 추가 영입했다.우선 프렌차이즈 스타 김영욱과 2020년까지 연장계약을 했다.전남 유스 출신인 김영욱은 당초 계약기간이 1년 남아 있었지만 전남의 프렌차이즈 스타로 팀의 중추적인 역할을 하고 있어 2020년까지 연장계약을 체결하게 됐다.전남은 이어 측면 공격력 강화를 위해 포항에서 활약한 완델손을 1년 임대 영입했다.완델손은 왼발 크로스가 정확하고 화려한 테크닉을 이용한 측면 공격력이 뛰어난 선수이다.또한 윙 포워드는 물론 공격형 미드필더 측면 수비수로도 뛸 수 있는 멀티 플레이어다.공격력 강화를 위해 페체신 대체자로 중국 리그에서 활약한 하태균도 영입했다.하태균은2007년 드래프트 1순위로 수원 삼성에 입단해 K리그 신인상을 수상한 공격수로 2014년까지 K리그 통산 135경기 30골 7도움을 기록했다.2015년 중국으로 이적 연변에서 26골을 기록 득점왕에 오르며 팀을 슈퍼리그로 승격시켰다.또한 2017년에는 바오딩으로 이적해 팀 내 최다골인 14골을 기록할 정도로 득점력이 뛰어난 선수다.이와 함께 박태준 이경렬 윤동민도 영입하며 올 시즌을 준비하고 있다.',
  '0'],
 ['11 24일 매실 감 참다래 등

In [10]:
# modified_dset_val = []
# it_cnt = 0
# eco_cnt = 0
# for news in tqdm(dataset1['valid']):
#      if news['category'] == '종합':
#           continue
#      temp = []
#      temp.append(news['article'])
#      if news['category'] == '경제' or news['category'] == '기업' or news['category'] == 'IT,과학':
#           if news['category'] == '경제':
#               eco_cnt +=1
#           if news['category'] == 'IT,과학':
#               it_cnt +=1
#           if eco_cnt >= 1000:
#                continue
#           if it_cnt >= 1000:
#                continue
#           temp.append('1')
#      else:
#          temp.append('0')
#      modified_dset_val.append(temp)

Counter({'종합': 13666, 'IT,과학': 10997, '경제': 3403, '사회': 1503, '스포츠': 392, '정치': 161})

In [11]:
modified_dset_val = []
it_cnt = 0
eco_cnt = 0
for news in tqdm(dataset1['valid']):
    if news['category'] == '종합':
        continue
    temp = []
    temp.append(news['article'])
    if news['category'] == '경제':
        eco_cnt +=1
        if eco_cnt >= 1000:
            continue
        temp.append('1')
    elif news['category'] == '기업':
        temp.append('2')
    elif news['category'] == 'IT,과학':
        it_cnt +=1
        if it_cnt >= 1000:
            continue      
        temp.append('3')  
    else:
         temp.append('0')
    modified_dset_val.append(temp)

100%|██████████| 30122/30122 [00:04<00:00, 6904.79it/s]


In [12]:
len(modified_dset_val)

4054

In [13]:
modified_dset_test = []
for news in tqdm(dataset1['valid']):
    temp = []
    if news['category'] == '종합':
        temp.append(news['article'])
        temp.append('0')
        modified_dset_test.append(temp)

100%|██████████| 30122/30122 [00:04<00:00, 6917.38it/s]


다운로드한 기사들과 라벨을 TSV 파일로 저장함

In [14]:
import csv

fields = ['article','label']

with open("dataset_train.tsv", "w", encoding="utf-8") as f:
    tw = csv.writer(f, delimiter="\t")

    tw.writerow(fields)
    tw.writerows(modified_dset)

In [15]:
with open("dataset_val.tsv", "w", encoding="utf-8") as f:
    tw = csv.writer(f, delimiter="\t")

    tw.writerow(fields)
    tw.writerows(modified_dset_val)

In [16]:
with open("dataset_test.tsv", "w", encoding="utf-8") as f:
    tw = csv.writer(f, delimiter="\t")

    tw.writerow(fields)
    tw.writerows(modified_dset_test)

In [17]:
dataset_train = nlp.data.TSVDataset('./dataset_train.tsv', field_indices=[0,1], num_discard_samples=1)
dataset_val = nlp.data.TSVDataset('./dataset_val.tsv', field_indices=[0,1], num_discard_samples=1)

# 토큰화, 패딩, 어텐션 마스크, 배치 및 데이터로더 설정

In [18]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [19]:
## Setting parameters
max_len = 500
batch_size = 4
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [20]:
# KoBERT 토크나이저를 불러들임
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
# BERTDataset 클래스 이용, TensorDataset으로 만들어주기
# index 0: sentence, index 1: label
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_val, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
# Batch 및 Dataloader 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=4)

# KoBERT 모델링

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2, ## class가 0과 1 두 개인 binary classification
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
# Optimizer, Scheduler, linear warmup and decay
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 기존 모델로 Valid

In [ ]:
# import copy
# PATH = './weights/'
# best_acc = 0.0
# model = torch.load('/opt/ml/kobert-news-classification/model.pt').to(device)  # 전체 모델을 통째로 불러옴, 클래스 선언 필수

# train_acc = 0.0
# val_acc = 0.0
# model.eval()
# for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):
#     token_ids = token_ids.long().to(device)
#     segment_ids = segment_ids.long().to(device)
#     valid_length= valid_length
#     label = label.long().to(device)
#     out = model(token_ids, valid_length, segment_ids)
#     val_acc += calc_accuracy(out, label)
# print("validation acc {}".format(val_acc / (batch_id+1)))

# Train, Valid

In [ ]:
import copy
best_acc = 0.0
PATH = './weights/'

for e in range(num_epochs):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        val_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, val_acc / (batch_id+1)))
    if best_acc < val_acc / (batch_id+1):
        best_acc = val_acc / (batch_id+1)
        best_model_weight = copy.deepcopy(model)
        torch.save(best_model_weight, PATH + 'multi_label_model.pt')  # 전체 모델 저장
    print("epoch {} best acc {}".format(e+1, best_acc))

/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/9259 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

# 테스트

In [ ]:
model = torch.load(PATH + 'model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수

In [ ]:
# new_test = nlp.data.TSVDataset('./dataset_test.tsv', field_indices=[0,1], num_discard_samples=1)
# test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
# test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
# for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
#   token_ids = token_ids.long().to(device) 
#   segment_ids = segment_ids.long().to(device) 
#   valid_length= valid_length 
#   out = model(token_ids, valid_length, segment_ids)
#   prediction = out.cpu().detach().numpy().argmax()
#   print(str(batch_id) + "번째 문장의 분류 예측값은 " + str(prediction) + "입니다.")

/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/13666 [00:00<?, ?it/s]

0번째 문장의 분류 예측값은 0입니다.
1번째 문장의 분류 예측값은 0입니다.
2번째 문장의 분류 예측값은 0입니다.
3번째 문장의 분류 예측값은 0입니다.
4번째 문장의 분류 예측값은 0입니다.
5번째 문장의 분류 예측값은 0입니다.
6번째 문장의 분류 예측값은 0입니다.
7번째 문장의 분류 예측값은 0입니다.
8번째 문장의 분류 예측값은 0입니다.
9번째 문장의 분류 예측값은 0입니다.
10번째 문장의 분류 예측값은 0입니다.
11번째 문장의 분류 예측값은 1입니다.
12번째 문장의 분류 예측값은 0입니다.
13번째 문장의 분류 예측값은 0입니다.
14번째 문장의 분류 예측값은 0입니다.
15번째 문장의 분류 예측값은 0입니다.
16번째 문장의 분류 예측값은 0입니다.
17번째 문장의 분류 예측값은 0입니다.
18번째 문장의 분류 예측값은 0입니다.
19번째 문장의 분류 예측값은 0입니다.
20번째 문장의 분류 예측값은 0입니다.
21번째 문장의 분류 예측값은 0입니다.
22번째 문장의 분류 예측값은 1입니다.
23번째 문장의 분류 예측값은 1입니다.
24번째 문장의 분류 예측값은 1입니다.
25번째 문장의 분류 예측값은 0입니다.
26번째 문장의 분류 예측값은 0입니다.
27번째 문장의 분류 예측값은 0입니다.
28번째 문장의 분류 예측값은 0입니다.
29번째 문장의 분류 예측값은 0입니다.
30번째 문장의 분류 예측값은 0입니다.
31번째 문장의 분류 예측값은 0입니다.
32번째 문장의 분류 예측값은 0입니다.
33번째 문장의 분류 예측값은 1입니다.
34번째 문장의 분류 예측값은 0입니다.
35번째 문장의 분류 예측값은 0입니다.
36번째 문장의 분류 예측값은 0입니다.
37번째 문장의 분류 예측값은 0입니다.
38번째 문장의 분류 예측값은 0입니다.
39번째 문장의 분류 예측값은 0입니다.
40번째 문장의 분류 예측값은 0입니다.
41번째 문장의 분류 예측값은 0입니다.
42번째 문장의 분류 예측값은 0입니다.
43번째 문장의 분류 예측값은 1입니다

In [ ]:
#extractive
from pororo import Pororo
summ = Pororo(task="summarization", model="extractive", lang="ko")
#요약할 txt입력
summ(pre_dart['엑세스바이오'])
